In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# data split and cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

#algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Load dataset and define features
url = "PIRUS_May2020/PIRUS_Public_May2020.csv"
attributes = []
dataset = pd.read_csv(url, attributes)
attributes = dataset.iloc[:, [0,1]]
print(attributes)

#filtering attributes and values to select what are needed


# Split-out validation dataset to a 8:2 ratio. 80% for training, 20% for testing
array = dataset.values
print(array)
X = array[:,0:4]
y = array[:,4]
# print(X)
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

# Spot Check Algorithms - compare different models to pick the best model
# does not need to be included in the final model
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
# Compare Algorithms
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

# Make predictions on validation dataset
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

      Subject_ID Loc_Plot_State1
0           1178        Virginia
1           2010        Delaware
2           2011   Massachusetts
3           2051           Texas
4           2063      Washington
...          ...             ...
2221        7781        New York
2222        7157       Minnesota
2223        7476    Pennsylvania
2224        7161  South Carolina
2225        8341            Utah

[2226 rows x 2 columns]
